# Tair

>[Tair](https://www.alibabacloud.com/help/en/tair/latest/what-is-tair) 是由 `Alibaba Cloud` 开发的一款云原生内存数据库服务。
它提供了丰富的数据模型和企业级功能，以支持您的实时在线场景，同时保持与开源 `Redis` 的完全兼容。`Tair` 还推出了基于新型非易失性内存 (NVM) 存储介质的持久性内存优化实例。

本 Notebook 展示了如何使用与 `Tair` 向量数据库相关的в功能。

您需要通过 `pip install -qU langchain-community` 安装 `langchain-community` 才能使用此集成。

要运行，您应该有一个正在运行的 `Tair` 实例。

In [ ]:
from langchain_community.embeddings.fake import FakeEmbeddings
from langchain_community.vectorstores import Tair
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../../how_to/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = FakeEmbeddings(size=128)

使用 `TAIR_URL` 环境变量连接到 Tair
```
export TAIR_URL="redis://{username}:{password}@{tair_address}:{tair_port}"
```

或者通过关键字参数 `tair_url` 进行连接。

然后将文档和 embeddings 存储到 Tair 中。

In [ ]:
tair_url = "redis://localhost:6379"

# drop first if index already exists
Tair.drop_index(tair_url=tair_url)

vector_store = Tair.from_documents(docs, embeddings, tair_url=tair_url)

查询相似文档。

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = vector_store.similarity_search(query)
docs[0]

Tair Hybrid Search 索引构建

In [ ]:
# drop first if index already exists
Tair.drop_index(tair_url=tair_url)

vector_store = Tair.from_documents(
    docs, embeddings, tair_url=tair_url, index_params={"lexical_algorithm": "bm25"}
)

Tair 混合搜索

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
# hybrid_ratio: 0.5 hybrid search, 0.9999 vector search, 0.0001 text search
kwargs = {"TEXT": query, "hybrid_ratio": 0.5}
docs = vector_store.similarity_search(query, **kwargs)
docs[0]